ЗАДАНИЕ

1. Посчитать показатели Retention Rate седьмого дня и ARPPU за два периода: до и после внедрения фичи.
2. Сформировать выводы по анализу.

Считаем Retention Rate 7 дня за период до внедрения фичи. 

In [73]:
%matplotlib inline
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras 

Создаем датафрейм

In [74]:
def getRegisteredUsersCount():
    query = '''SELECT r.registration_date, count(distinct(r.user_id)) as registered_users_count
    FROM case9.registration r
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

registered_users_count = pd.DataFrame(getRegisteredUsersCount())
registered_users_count['registration_date'] = pd.to_datetime(registered_users_count['registration_date'])
def getActiveUsersCountWithCohorts():
    query = '''SELECT ac.activity_date, r.registration_date, count(distinct(ac.user_id)) as active_users_count
    FROM case9.user_activity ac
    LEFT JOIN case9.registration r ON r.user_id = ac.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data


In [75]:
active_users_count_with_cohorts = pd.DataFrame(getActiveUsersCountWithCohorts())
for col in ['activity_date','registration_date']:
    active_users_count_with_cohorts[col] = pd.to_datetime(active_users_count_with_cohorts[col])
retention_table = active_users_count_with_cohorts.merge(registered_users_count,on=['registration_date'],how='left')#рассчитываем lifetime для каждой строки
retention_table['lifetime'] = retention_table['activity_date'] - retention_table['registration_date']
retention_table['lifetime'] = retention_table['lifetime']/np.timedelta64(1,'D')
retention_table['lifetime'] = retention_table['lifetime'].astype(int) # Приведем тип к целому числу
retention_table.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime
0,2019-05-01,2019-05-01,2590,6516,0
1,2019-05-02,2019-05-01,2146,6516,1
2,2019-05-02,2019-05-02,2049,5309,0
3,2019-05-03,2019-05-01,1637,6516,2
4,2019-05-03,2019-05-02,1817,5309,1


Добавим в таблицу данные по выручке в разрезе дней.

In [76]:
def getRevenue():
    query = '''SELECT re.date, r.registration_date, count(distinct(re.user_id)) as users_count_with_revenue, sum(re.revenue) as revenue
    FROM case9.user_revenue re
    LEFT JOIN case9.registration r ON r.user_id = re.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

revenue = pd.DataFrame(getRevenue())

for col in ['date','registration_date']:
    revenue[col] = pd.to_datetime(revenue[col])
    
revenue = revenue.rename(columns={'date':'activity_date'})
retention_table_with_revenue = retention_table.merge(revenue,on=['registration_date','activity_date'],how='left')
for col in ['revenue','users_count_with_revenue']:
    retention_table_with_revenue[col] = retention_table_with_revenue[col].fillna(0)
retention_table_with_revenue['users_count_with_revenue'] = retention_table_with_revenue['users_count_with_revenue'].astype(int)# Приведем тип к целому числу
retention_table_with_revenue.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,2590,6516,0,1672,2357.580
1,2019-05-02,2019-05-01,2146,6516,1,1420,514.679
2,2019-05-02,2019-05-02,2049,5309,0,1367,1629.140
3,2019-05-03,2019-05-01,1637,6516,2,1048,390.952
4,2019-05-03,2019-05-02,1817,5309,1,1164,533.608


Далее из общего датафрема отфильтруем данные по lifetime 7 на период до запуска фичи (май 2019)

In [77]:
retention_7_may_table = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>="2019-05-01"][retention_table_with_revenue['registration_date']<"2019-06-01"]
retention_7_may_table.head()

<ipython-input-77-7f2647d2bdcd>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  retention_7_may_table = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>="2019-05-01"][retention_table_with_revenue['registration_date']<"2019-06-01"]


,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
28,2019-05-08,2019-05-01,953,6516,7,616,63.9065
37,2019-05-09,2019-05-02,745,5309,7,490,313.7230
47,2019-05-10,2019-05-03,895,6266,7,571,110.7260
58,2019-05-11,2019-05-04,1186,8639,7,750,124.8590
70,2019-05-12,2019-05-05,1358,9787,7,885,174.9240


In [78]:
print ('Retention 7 дня для пользователей, привлеченных до релиза = {:.2%}'.format(retention_7_may_table['active_users_count'].sum() / retention_7_may_table['registered_users_count'].sum()))

Retention 7 дня для пользователей, привлеченных до релиза = 12.93%


Далее из общего датафрема отфильтруем данные по lifetime 7 на период после запуска фичи (июнь 2019)

In [79]:
retention_7_june_table = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>="2019-06-01"][retention_table_with_revenue['registration_date']<"2019-07-01"]
retention_7_june_table.head()

<ipython-input-79-d06bb8139afa>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  retention_7_june_table = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>="2019-06-01"][retention_table_with_revenue['registration_date']<"2019-07-01"]


,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
736,2019-06-08,2019-06-01,545,4833,7,365,176.3160
767,2019-06-09,2019-06-02,683,5255,7,461,193.8390
798,2019-06-10,2019-06-03,484,4193,7,320,27.7548
829,2019-06-11,2019-06-04,511,4194,7,320,54.9610
860,2019-06-12,2019-06-05,472,3998,7,301,63.1554


In [80]:
print ('Retention 7 дня для пользователей, привлеченных после релиза = {:.2%}'.format(retention_7_june_table['active_users_count'].sum() / retention_7_june_table['registered_users_count'].sum()))

Retention 7 дня для пользователей, привлеченных после релиза = 12.56%


Заметна динамика на понижение показателя до и после релиза.
Определим % снижения.

In [81]:
print ('Retention 7 дня для пользователей снизился на {:.2%}'.format(12.93 / 12.56-1))

Retention 7 дня для пользователей снизился на 2.95%


Retention 7 снизился незначительно. возможно на снижение повлияли внешние факторы не связанные с выпуском фичи.
Расчитаем ARPPU до и после резиза.

In [82]:
print ('ARPPU до релиза = ', round(retention_table_with_revenue['revenue'][retention_table_with_revenue['registration_date']>="2019-05-01"]
                                   [retention_table_with_revenue['registration_date']<"2019-06-01"].sum() / 
                                   retention_table_with_revenue['users_count_with_revenue'][retention_table_with_revenue['registration_date']>="2019-05-01"]
                                   [retention_table_with_revenue['registration_date']<"2019-06-01"].sum(),2))

print ('ARPPU после релиза = ', round(retention_table_with_revenue['revenue'][retention_table_with_revenue['registration_date']>="2019-06-01"]
                                   [retention_table_with_revenue['registration_date']<"2019-07-01"].sum() / 
                                   retention_table_with_revenue['users_count_with_revenue'][retention_table_with_revenue['registration_date']>="2019-06-01"]
                                   [retention_table_with_revenue['registration_date']<"2019-07-01"].sum(),2))

ARPPU до релиза =  0.31
ARPPU после релиза =  0.39


Очевидно, что доход с платящих клиентов вырос. Определим на сколько %

In [83]:
print ('Доход после выпуска фичи с платящего клиента увеличился на {:.2%}'.format(0.39 / 0.31-1))

Доход после выпуска фичи с платящего клиента увеличился на 25.81%


Выводы: Несмотря на на снижение количества привлеченных клиентов ( на 3%), общий доход после выпуска фичи увеличился на 25%. Поскольку фича привела к положительной динамике, что говорит о ее эффективности, ее можно принять в дальнейшую работу. Тем не менее трбуются дальнейшие наблюдения за показателями на более длительном промежутке времени.